In [174]:
from sklearn.feature_extraction import DictVectorizer
import xgboost as xgb
import pandas as pd
import numpy as np
import pickle
import os
import re
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot
from hdfs import InsecureClient
import string
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
from sklearn.metrics import accuracy_score, precision_score,recall_score,f1_score
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from collections import OrderedDict
from sklearn.linear_model import LogisticRegression
client_hdfs = InsecureClient("http://10.65.31.100:50070")

In [10]:
def remove_punctuations(text):
    rmvSpcchar2 = re.compile("[\t**' ]")
    rmvSpcchar1 = re.compile("[/(){}\[\]\|$*\-]")
    rmvSpcchar = re.compile("[/(){}\[\]\|&@,;_'.-]")
    rmvBadSym = re.compile('[^\d+\0-9a-z#+_]')
    rmvNumbers = re.compile('\d+')
    text = re.sub(rmvSpcchar, ' ', text)
    text = re.sub(rmvBadSym, ' ', text)
    text = re.sub(rmvNumbers, ' ', text)
    return text


def is_vowel(char):

    if char in ('a', 'e', 'i', 'o', 'u'):
        return'1'
    else:
        return'0' 

In [2]:
def data_read(url):
    with client_hdfs.read(url , encoding = 'utf-8') as reader:
        if client_hdfs.content(url)["spaceConsumed"]>1000:
            df_input = pd.read_csv(reader,names=["name"])
            return df_input

In [3]:
with client_hdfs.read( "/user/dataplatform/indicium/temp/NM/newregistration12345/part-00000" , encoding = 'utf-8') as reader:
    df = pd.read_csv(reader)   

In [6]:
df=df[df["guid"].notna()]
df.fillna(' ', inplace=True)

In [8]:
imsMaping = df[["guid","imsId"]].reset_index(drop=True)
display(imsMaping.shape)
imsMaping = imsMaping.drop_duplicates(subset={'guid'})
display(imsMaping.shape)

(23486164, 2)

(23151963, 2)

In [9]:
df.drop_duplicates(subset={'guid','displayName','firstName','middleName','lastName','gender'},inplace=True)
df = df.replace({'default':' '})
df['FullName'] = df['displayName']+ " " + df['firstName'] + " " + df['middleName'] + " " +df['lastName']
df["FullNameCleaned"] = df["FullName"].apply(lambda x: remove_punctuations(x))
df['Cleaned Name']=df['FullNameCleaned'].apply(lambda x: list(filter(None,OrderedDict.fromkeys(x.split(" ")))))

In [15]:
df.drop(['firstName','middleName','lastName'],axis=1,inplace=True)
df['Counts'] = df['Cleaned Name'].apply(lambda x: len(x))
index_names = df[ df["Counts"]==0 ].index
df.drop(index_names, inplace = True)

In [16]:
df['FirstName'] = df['Cleaned Name'].apply(lambda x: x[0])
df["FirstNameLength"] = df["FirstName"].apply(lambda x: len(x))
df = df[df["FirstNameLength"]>=3]
df["flagss"]=df["FirstName"].apply(lambda x: x.isalpha())
df.drop(df[df['gender']=='o'].index, inplace=True) 
df.drop(df[df["flagss"]==False].index, inplace=True)  
df['LastName'] = df['Cleaned Name'].apply(lambda x: x[1] if len(x)>1 else "")
df['ExtendedName'] = df['Cleaned Name'].apply(lambda x: " ".join(x[2:]) if len(x)>2 else "")

In [25]:
directRecommendation = df[df["FirstName"].isin(['mr','mrs','smt','kumar','kumari','miss','shri','mohd'])]
df.drop(directRecommendation.index,inplace=True)
toRemove = df[df["FirstName"].isin(['dr','md','all','gaming','technical','royal','the','day','unknown','include'])]
df.drop(toRemove.index,inplace=True)
df = df.reset_index(drop=True)

In [27]:
FnFreq = df["FirstName"].value_counts().reset_index()
fnList = FnFreq["index"].tolist()
LnFreq = df["LastName"].value_counts().reset_index()
lnList = LnFreq["index"].tolist()
nameMerge = FnFreq.merge(LnFreq, on ="index", how="inner")

In [28]:
finalDf = df[['gender',"FirstName"]].reset_index(drop=True)
finalDf["Vowel_Present"] = finalDf["FirstName"].apply(lambda x: is_vowel(x[-1]))

In [33]:
gender_count=finalDf[['gender','Vowel_Present']].value_counts()
gender_count

In [35]:
Females = finalDf[finalDf["gender"]=="f"]
Males = finalDf[finalDf["gender"]=="m"]

In [36]:
Females_0=Females[Females['Vowel_Present']=='0'] #no vowel female gender
Females_0

,gender,FirstName,Vowel_Present
3,f,ramesh,0
32,f,thanush,0
38,f,sagar,0
54,f,shekh,0
56,f,rangrej,0
...,...,...,...
22495346,f,anees,0
22495704,f,meher,0
22495782,f,suman,0
22496010,f,jasmeet,0


In [37]:
Males_1=Males[Males['Vowel_Present']=='1']  #with vowel male gender
Males_1

,gender,FirstName,Vowel_Present
2,m,kranthi,1
4,m,satti,1
7,m,kandiri,1
20,m,devendra,1
27,m,ravindra,1
...,...,...,...
22496086,m,santu,1
22496087,m,rama,1
22496089,m,saloni,1
22496095,m,pasunuri,1


In [38]:
df['gender'].value_counts()

m    19369816
f     3126287
Name: gender, dtype: int64

In [39]:
def features(name):
    name=str(name)
    name = (name.lower()).split()[0]
    return {
        'first-letter': name[0], # First letter
        'first2-letters': name[0:2], # First 2 letters
        'first3-letters': name[0:3], # First 3 letters
        'first4-letters': name[0:4],
        'last-letter': name[-1],
        'vowel': is_vowel(name[-1]),
        'last2-letters': name[-2:],
        'last3-letters': name[-3:],
    }

In [41]:
finalDf['binaryGender'] = finalDf['gender'].replace({'m':0,'f':1})
X=finalDf.drop(['gender','binaryGender','Vowel_Present'],axis=1)
y=finalDf['binaryGender']
X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size=0.25, random_state=42)

In [47]:
X_train['FirstName']=X_train['FirstName'].astype(str)
X_train = X_train.reset_index(drop=True)
features = np.vectorize(features)
namesData=features(X_train['FirstName'])
vectorizer = DictVectorizer()
vect = vectorizer.fit_transform(namesData)

In [49]:
clf = LogisticRegression(solver='lbfgs',max_iter=100)
clf.fit(vect, y_train)

array([0, 1])

In [55]:
Testresult=features(X_test['FirstName'])
vecttest = vectorizer.transform(Testresult)
result= clf.predict(vecttest)
y_test['prediction'] = result
y_testL = y_test.tolist()

In [172]:
print(accuracy_score(y_testL, result))
print(precision_score(y_testL, result, average='macro'))
print(recall_score(y_testL, result, average='macro'))
print(f1_score(y_testL, result, average='macro'))

0.864355001203764
0.7116251381431047
0.5586492673916494
0.5713889824355468
